# Qiskit Multirunner

### Imports

In [11]:
import json
import csv

import qiskit
from qiskit import QuantumCircuit, QuantumRegister
import qiskit_aer
from mqt import ddsim
from qiskit_ibm_runtime import SamplerV2 as Sampler, QiskitRuntimeService

from qiskit_aer import AerSimulator
from qiskit_qulacs import QulacsProvider

### Sample Circuit

In [3]:
def get_sample_circuit() -> QuantumCircuit:
    qubits = QuantumRegister(1)
    qc = QuantumCircuit(qubits)
    qc.h(qubits[0])
    qc.measure_all()
    return qc

In [4]:
# Display our sample circuit
qc = get_sample_circuit()
qc.draw()

┌───┐ ░ ┌─┐
    q1: ┤ H ├─░─┤M├
        └───┘ ░ └╥┘
meas: 1/═════════╩═
                 0

### Qiskit AER Execution

In [5]:
def run_on_aer(qc: QuantumCircuit):
    backend = AerSimulator()
    result = backend.run(qc).result()
    return result
run_on_aer(get_sample_circuit())

Result(backend_name='aer_simulator', backend_version='0.16.3', qobj_id='', job_id='10a30685-19cc-42b4-adff-a8a706059059', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 526, '0x1': 498}), header=QobjExperimentHeader(creg_sizes=[['meas', 1]], global_phase=0.0, memory_slots=1, n_qubits=1, name='circuit-163', qreg_sizes=[['q2', 1]], metadata={}), status=DONE, seed_simulator=3360387923, metadata={'num_bind_params': 1, 'runtime_parameter_bind': False, 'parallel_state_update': 4, 'parallel_shots': 1, 'sample_measure_time': 0.001029526, 'noise': 'ideal', 'batched_shots_optimization': False, 'remapped_qubits': False, 'active_input_qubits': [0], 'device': 'CPU', 'time_taken': 0.014270702, 'measure_sampling': True, 'num_clbits': 1, 'max_memory_mb': 8192, 'input_qubit_map': [[0, 0]], 'num_qubits': 1, 'method': 'stabilizer', 'required_memory_mb': 0, 'fusion': {'enabled': False}}, time_taken=0.014270702)], date=2025-03-28T11:

### Qiskit MQT Execution

In [6]:
def run_on_mqt(qc: QuantumCircuit):
    backend = ddsim.DDSIMProvider().get_backend("qasm_simulator")
    result = backend.run(qc).result()
    return result

run_on_mqt(get_sample_circuit())

Result(backend_name='qasm_simulator', backend_version='1.24.0', qobj_id='None', job_id='782f1cec-cfbd-4c03-8413-4dffd718c45d', success=True, results=[ExperimentResult(shots=1024, success=True, meas_level=2, data=ExperimentResultData(counts={'0x0': 513, '0x1': 511}, time_taken=0.2624969482421875), header=DDSIMHeader(name='circuit-164', n_qubits=1, memory_slots=1, global_phase=0.0, creg_sizes=[('meas', 1)], clbit_labels=[('meas', 0)], qreg_sizes=[('q3', 1)], qubit_labels=[('q3', 0)]), status=DONE, seed=-1, metadata={})], date=None, status=COMPLETED, header=None, time_taken=0.2740018367767334)

### Qiskit Qulacs Execution

In [7]:
def run_on_qulacs(qc: QuantumCircuit):
    backend = QulacsProvider().get_backend('qulacs_simulator')
    result = backend.run(qc).result()
    return result

run_on_qulacs(get_sample_circuit())

/var/folders/0q/dj3wgc5576l38_bq_5tdyvf40000gn/T/ipykernel_1252/180786299.py:2: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = QulacsProvider().get_backend('qulacs_simulator')
/Users/mews/Documents/repos/qiskit-multirunner/.conda/lib/python3.11/site-packages/qiskit_qulacs/adapter.py:246: UserWarning: qiskit_qulacs.adapter: The Barrier instruction is not supported by Qiskit-Qulacs and has not been added to the circuit.
  warnings.warn(
/Users/mews/Documents/repos/qiskit-multirunner/.conda/lib/python3.11/site-packages/qiskit_qulacs/adapter.py:246: UserWarning: qiskit_qulacs.adapter: The Measure instruction is not supported by Qiskit-Qulacs and has not bee

Result(backend_name='qulacs_simulator', backend_version='0.1.0', qobj_id='0', job_id='2653854d-b590-43d2-a408-ba5fb85d5977', success=True, results=[ExperimentResult(shots=0, success=True, meas_level=2, data=ExperimentResultData(statevector=[0.70710678+0.j 0.70710678+0.j]), status=JobStatus.DONE)], date=None, status=JobStatus.DONE, header=None, time_taken=0.0008161067962646484)

In [8]:
def log_1000_simulations_per_sumulator():
    samples = []
    circuit = get_sample_circuit()
    for i in range(0,999):
        execution_dict = run_on_aer(circuit).to_dict()
        samples.append(execution_dict)
    for i in range(0,999):
        execution_dict = run_on_mqt(circuit).to_dict()
        samples.append(execution_dict)
    for i in range(0,999):
        execution_dict = run_on_qulacs(circuit).to_dict()
        samples.append(execution_dict)
    return samples

def executions_to_csv(samples: list, target_file: str):
    file = open(target_file, 'r+')
    #get a sample of the dataset for write parameters
    sample = samples[0]
    #write the type header
    header = ','.join(sample.keys())
    file.write(header)

    writer = csv.DictWriter(file, fieldnames=list(sample.keys()))

    for i in samples:
        writer.writerow(i)

    file.close()

    pass

execs = log_1000_simulations_per_sumulator()
executions_to_csv(execs, '../output/simulations.csv')

/var/folders/0q/dj3wgc5576l38_bq_5tdyvf40000gn/T/ipykernel_1252/180786299.py:2: DeprecationWarning: The class ``qiskit.providers.provider.Provider`` is deprecated as of qiskit 1.1. It will be removed no earlier than 3 months after the release date. The abstract Provider and ProviderV1 classes are deprecated and will be removed in 2.0. You can just remove it as the parent class and a `get_backend` method that returns the backends from `self.backend`.
  backend = QulacsProvider().get_backend('qulacs_simulator')
/Users/mews/Documents/repos/qiskit-multirunner/.conda/lib/python3.11/site-packages/qiskit_qulacs/adapter.py:246: UserWarning: qiskit_qulacs.adapter: The Barrier instruction is not supported by Qiskit-Qulacs and has not been added to the circuit.
  warnings.warn(
/Users/mews/Documents/repos/qiskit-multirunner/.conda/lib/python3.11/site-packages/qiskit_qulacs/adapter.py:246: UserWarning: qiskit_qulacs.adapter: The Measure instruction is not supported by Qiskit-Qulacs and has not bee

# Real-hardware execution

Configure the following with your account in order to execute jobs in actual IBM hardware, it implies a compute time cost, so you should probably be cautious about running this multiple times.

In [ ]:
#please put your account info here
QiskitRuntimeService.save_account(
  token='',
  channel="ibm_quantum", # `channel` distinguishes between different account types
  overwrite=True # To change the account again for every execution, useful if you're using multiple tokens
)

def run_on_hw(qc: QuantumCircuit):
    service = QiskitRuntimeService()
    # Get a backend
    backend = service.least_busy(operational=True, simulator=False)
    
    # Define Sampler
    sampler = Sampler(mode=backend)
    
    # Run calculation
    job = sampler.run([qc])
    
    result = job.result()
    return result

run_on_hw(get_sample_circuit())

IBMInputValueError: 'The instruction h on qubits (0,) is not supported by the target system. Circuits that do not match the target hardware definition are no longer supported after March 4, 2024. See the transpilation documentation (https://docs.quantum.ibm.com/guides/transpile) for instructions to transform circuits and the primitive examples (https://docs.quantum.ibm.com/guides/primitives-examples) to see this coupled with operator transformations.'